In [120]:
import requests
import unicodedata

destinatario = '08187168000160'
api_brasil = requests.get(f'https://brasilapi.com.br/api/cnpj/v1/{destinatario}')
destino = api_brasil.json()
print(destino)

{'uf': 'PR', 'cep': '87014010', 'qsa': [{'pais': None, 'nome_socio': 'ERIKE LEITE DE ALMEIDA', 'codigo_pais': None, 'faixa_etaria': 'Entre 41 a 50 anos', 'cnpj_cpf_do_socio': '***493259**', 'qualificacao_socio': 'Presidente', 'codigo_faixa_etaria': 5, 'data_entrada_sociedade': '2007-12-03', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 16, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante_legal': 0}, {'pais': None, 'nome_socio': 'RODRIGO RAMALHO XAVIER', 'codigo_pais': None, 'faixa_etaria': 'Entre 41 a 50 anos', 'cnpj_cpf_do_socio': '***885519**', 'qualificacao_socio': 'Diretor', 'codigo_faixa_etaria': 5, 'data_entrada_sociedade': '2016-12-08', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'codigo_qualificacao_socio': 10, 'qualificacao_representante_legal': 'Não informada', 'codigo_qualificacao_representante

In [121]:
def remover_acentos(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def pegar_codigo_cidade(estado, cidade):
    
    # URL api Brasil para ibge
    url = f'https://brasilapi.com.br/api/ibge/municipios/v1/{estado}'
    requisicao = requests.get(url)
    dados_empresa = requisicao.json()
    for dicionario in dados_empresa:
        if cidade == remover_acentos(dicionario.get('nome')):
            cidade = dicionario.get('nome'), 
            codigo_cidade = dicionario.get('codigo_ibge')
    return cidade[0], codigo_cidade

In [122]:
destino.get('logradouro')

'CARNEIRO LEAO'

In [123]:

itens_produtos = {
    "descricao": "Produto 1",
    "valor_unitario": 120
}

# Cálculo do valor total da NFe
quantidade = 5
valor_unitario = itens_produtos.get('valor_unitario')
valor_pagamento = float(valor_unitario * quantidade)


valor_unitario = float(valor_unitario)

In [124]:
valor_unitario

120.0

In [125]:
valor_pagamento

600.0

In [126]:
pegar_codigo_cidade(destino.get('uf'), destino.get('municipio'))

('MARINGÁ', '4115200')

In [127]:
# Como emitir 


cnpj = '15436220000130'
link = f'https://api.sandbox.plugnotas.com.br/nfe'

headers = {
    'x-api-key': '2da392a6-79d2-4304-a8b7-959572c7e44d'
}

dados_emissao = [{
    "emitente": {
        "cpfCnpj": cnpj,
        },
    
    "destinatario": {
        "cpfCnpj": destino.get('cnpj'),
        "endereco": {
            "bairro": destino.get('bairro'),
            "cep":destino.get('cep'),
            "codigoCidade" : pegar_codigo_cidade(destino.get('uf'), destino.get('municipio'))[1],
            "estado": destino.get('uf'),
            "logradouro": destino.get('logradouro'),
            "numero": destino.get('numero'),
            "tipoLogradouro": destino.get('descricao_tipo_de_logradouro'),            
        }
    },
    "itens": [{
        "descricao":  itens_produtos.get('descricao'),
        "ncm": "83100000",
        "cfop": "5102",
        "tributos": {
            "pis":{
                "cst": "99",
                "baseCalculo": {
                    "valor": 0.00
                },
                "aliquota": 0.00 
            },
            "cofins": {
                "cst": "99",
                "baseCalculo": {
                    "valor": 0.00
                },
                "aliquota": 0.00,
                "valor": 0.00,
                
            },
                    
        }, 
        
        "valorUnitario": {
             "comercial": float(valor_unitario),
             "tributavel": float(valor_unitario),
         }, 
    }],
   
    "pagamentos": {
        "aVista": True,
        "meio": "99",
        "valor": float(valor_pagamento),        
    },
},
]

emissao = requests.post(link, headers=headers, json=dados_emissao)

print(emissao.status_code)

print(emissao.text)

400
{"error":{"message":"Falha na validação do JSON de NFe","data":{"fields":{"NFe[undefined]":"Necessário enviar 'valorTroco' quando 'pagamentos.valor' for maior que o total da NFe"}}}}


In [128]:
float(valor_pagamento)

600.0

In [129]:
valor_unitario

120.0

In [156]:
# Solicitar Cancelamento da nota fiscal
idNota = '676edc26a22246160bafaef7'
url_canc = f"https://api.sandbox.plugnotas.com.br/nfe/{idNota}/cancelamento"
justificativa_msg = "Erro na emissão da Nota Fiscal eletrônica.  2024"

justificativa = {
    "justificativa": justificativa_msg
}

cancelar = requests.post(url_canc, headers=headers, json=justificativa)
print(cancelar.status_code)
print(cancelar.text)

200
{"message":"Cancelamento em processamento"}


In [157]:
# consultar status de nota cancelada
idNota = '676edc26a22246160bafaef7'
link_status = f"https://api.sandbox.plugnotas.com.br/nfe/{idNota}/cancelamento/status"

status = requests.get(link_status, headers=headers)
print(status.status_code)
print(status.text)

200
{"message":"Solicitação de cancelamento processada","data":{"justificativa":"Erro na emissão da Nota Fiscal eletrônica.  2024","respostaSefaz":"Evento registrado e vinculado a NF-e","xml":"https://api.sandbox.plugnotas.com.br/nfe/676edc26a22246160bafaef7/cancelamento/xml"}}


In [154]:
# Consultar resumo da nota
idNotaOrChaveOrProtocol = '676edc26a22246160bafaef7'
link_status = f"https://api.sandbox.plugnotas.com.br/nfe/{idNotaOrChaveOrProtocol}/resumo"

status = requests.get(link_status, headers=headers)
print(status.status_code)
print(status.text)

200
[{"id":"676edc26a22246160bafaef7","idIntegracao":"6ae1aadd-848d-47b0-81a0-1f94be3e7dbf","emissao":"27/12/2024","status":"CONCLUIDO","destinada":false,"emitente":"08187168000160","destinatario":"96603709000150","valor":0.1,"dataAutorizacao":"27/12/2024","numero":"1000013","chave":"41201008187168000160558050010000131609769080","protocolo":"141200000956123","mensagem":"Autorizado o uso da NF-e","xml":"https://api.sandbox.plugnotas.com.br/nfe/676edc26a22246160bafaef7/xml","pdf":"https://api.sandbox.plugnotas.com.br/nfe/676edc26a22246160bafaef7/pdf","cStat":100}]


In [153]:
# Baixar PDF da nota
idNota = '676edc26a22246160bafaef7'
url_pdf = f'https://api.sandbox.plugnotas.com.br/nfe/{idNota}/pdf'
requisicao = requests.get(url_pdf, headers=headers)
print(requisicao.status_code)
print(requisicao.content)

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

In [148]:
idNotaOrChaveOrProtocol = '676edc26a22246160bafaef7'
idNotaOrChaveOrProtocol

'676edc26a22246160bafaef7'

In [151]:
# Baixar XML da nota
url_xml = f'https://api.sandbox.plugnotas.com.br/nfe/{idNotaOrChaveOrProtocol}/xml'
requisicao = requests.get(url_xml, headers=headers)
print(requisicao.status_code)
print(requisicao.content)

200
b'<?xml version="1.0" encoding="UTF-8"?><nfeProc versao="4.00" xmlns="http://www.portalfiscal.inf.br/nfe"><NFe xmlns="http://www.portalfiscal.inf.br/nfe"><infNFe Id="NFe41200308187168000160558040007932721000000015" versao="4.00"><ide><cUF>41</cUF><cNF>00000001</cNF><natOp>VENDA DE MERCADORIA ADQ. DE TERCEIRO - PF E PJ NAO CONTRIBUI</natOp><mod>55</mod><serie>804</serie><nNF>793272</nNF><dhEmi>2020-03-05T07:50:42-03:00</dhEmi><tpNF>1</tpNF><idDest>1</idDest><cMunFG>4115200</cMunFG><tpImp>1</tpImp><tpEmis>1</tpEmis><cDV>5</cDV><tpAmb>2</tpAmb><finNFe>1</finNFe><indFinal>1</indFinal><indPres>1</indPres><procEmi>0</procEmi><verProc>5</verProc></ide><emit><CNPJ>08187168000160</CNPJ><xNome>Teste NOME</xNome><xFant>teste FANT.</xFant><enderEmit><xLgr>RUA DO POVO</xLgr><nro>711</nro><xBairro>jardim teste</xBairro><cMun>4115200</cMun><xMun>MARINGA</xMun><UF>PR</UF><CEP>79415000</CEP><cPais>1058</cPais><xPais>BRASIL</xPais><fone>4432222222</fone></enderEmit><IE>9044016688</IE><CRT>3</CRT></e

In [143]:
# Consultar notas por período
cnpj = '08187168000160'
url = f'https://api.sandbox.plugnotas.com.br/nfe/consulta/periodo?cpfCnpj={cnpj}&dataInicial=2024-12-01&dataFinal=2024-12-31'
nota_data = requisicao = requests.get(url, headers=headers)
print(nota_data.status_code)
print(nota_data.json())

200
{'hashProximaPagina': None, 'notas': [{'id': '676edc26a22246160bafaef7', 'idIntegracao': '6ae1aadd-848d-47b0-81a0-1f94be3e7dbf', 'emissao': '27/12/2024', 'status': 'CONCLUIDO', 'destinada': False, 'emitente': '08187168000160', 'destinatario': '96603709000150', 'valor': 0.1, 'dataAutorizacao': '27/12/2024', 'numero': '1000013', 'chave': '41201008187168000160558050010000131609769080', 'protocolo': '141200000956123', 'mensagem': 'Autorizado o uso da NF-e', 'xml': 'https://api.sandbox.plugnotas.com.br/nfe/676edc26a22246160bafaef7/xml', 'pdf': 'https://api.sandbox.plugnotas.com.br/nfe/676edc26a22246160bafaef7/pdf', 'cStat': 100}, {'id': '676d73f009fa3d4ea58615b7', 'idIntegracao': '5ae1aadd-848d-47b0-81a0-1f94be3e7dbf', 'emissao': '26/12/2024', 'status': 'CONCLUIDO', 'destinada': False, 'emitente': '08187168000160', 'destinatario': '96603709000150', 'valor': 0.1, 'dataAutorizacao': '26/12/2024', 'numero': '1000013', 'chave': '41201008187168000160558050010000131609769080', 'protocolo': '1

In [176]:
for indice, nota in enumerate(nota_data.json().get('notas')):
#     print(nota)
#     print(nota.get('id'))
    idNota = nota.get('id')
    url_pdf = f'https://api.sandbox.plugnotas.com.br/nfe/{idNota}/pdf'
    requisicao = requests.get(url_pdf, headers=headers)
#     print(requisicao.status_code)
#     print(requisicao.content)
    with open(fr"C:\Users\Positivo union\ApinotasFiscais/{indice}pdfnota.pdf", 'wb') as file:
        file.write(requisicao.content)
    
    


200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

...

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

200
b'%PDF-1.4\r\n%\x80\x81\x82\x83\r\n1 0 obj\r\n<< /Creator <>\r\n/CreationDate <443a32303230303431343131343635332d303327303027>\r\n/Title <>\r\n/Author <>\r\n/Producer <5265706f72744275696c646572>\r\n/Keywords <>\r\n/Subject <> >>\r\nendobj\r\n2 0 obj\r\n[/PDF /Text /ImageB /ImageC]\r\nendobj\r\n3 0 obj\r\n<< /Type /ExtGState\r\n/SA true\r\n>>\r\nendobj\r\n4 0 obj\r\n<< /Type /Catalog\r\n/Pages 5 0 R\r\n>>\r\nendobj\r\n6 0 obj\r\n<< /Length 5629     \r\n/Filter /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xb5=\xcbn\x1cG\x92w\x01\xfa\x87<\xec\x00\xb3\x0b\xab\x98\xef\x87o\xa5\xee\xa2]\x03\xb2\x9b\xeenj\r/\x16\x0b\x8eE{8\x90H\x9b\xa6F\xc0|\xda\xde\x06s0\xe6\xe0\xd3\xec~\xc6\xfe\xc3FD\xbd\xb2\xba\x9a"\xbb2%[\x04U\xec\x8a\x88\x8c\x8c\x8cw\x06\x7f~\xf9\xe2\x9b\x97/x\xc1Y\xf7w\xf3\x15\xfe\xdb\x19\xf6\xf1\xe5\x8b\xff\xf8O\xc6\xd9\xdb\x97/\x04<\xef\xfe\xde\xff\xf8\xf2\x854\x06>\xc1\x95c\xd6\xa9Bh\xee5\x0b\xa1\xe0\xf0\xc7\xb0`\n\xa9\x83\x97\xec\xfe\xfa\xe5\x8b\xd7\xc7\x03W\xc6"\x80`\x99\xd5\xa1\x10\

In [145]:
len(nota_data.json().get('notas'))

53

In [110]:
url_cidade = 'https://api.sandbox.plugnotas.com.br/nfse/cidades'
cidades = requisicao = requests.get(url_cidade, headers=headers)
print(cidades.status_code)


200


In [119]:
for cidade in cidades.json():
    if cidade.get('nome').lower() == 'recife':
        print(cidade)


{'id': 2611606, 'nome': 'Recife', 'uf': 'PE', 'padraoNacional': {'producao': False, 'homologacao': False}, 'padrao': 'TIPLAN', 'certificado': True, 'multiservicos': False, 'login': False, 'senha': False, 'lote': {}, 'sequencial': True, 'detalhesEmissao': {}, 'ambiente': {}, 'metodosDisponiveis': {}, 'token': False, 'detalhes': '', 'upload': False, 'descricao': 0, 'substituicao': True, 'informacoesComplementares': 0}


In [158]:
import os 
import PyPDF2


In [170]:
with open(fr"C:\Users\Positivo union\ApinotasFiscais/pdfnota.pdf", 'wb') as file:
    file.write(requisicao.content)

In [173]:
 with open(r'C:\Users\Positivo union\ApinotasFiscais\pdfnota.pdf', "rb") as doc:
    pdf = PyPDF2.PdfReader(doc)
    num_paginas = len(pdf.pages)
    pagina = pdf.pages[0]
    texto = pagina.extract_text()
    texto

In [174]:
print(texto)

NATUREZA DA OPERAÇÃO
INSCRIÇÃO ESTADUAL INSCRIÇÃO ESTADUAL SUB. TRIBUTARIA CNPJCHAVE DE ACESSO0 - Entrada
1 - Saída
Nº 
SÉRIE :1
000.793.272
VENDA DE MERCADORIA ADQ. DE TERCEIRO - PF E PJ NAO CONTRIBUI
9044016688 08.187.168/0001-604120  0308  1871  6800  0160  5580  4000  7932  7210  0000  0015DOCUMENTO AUXILIAR
DA NOTA FISCAL Teste NOME
RUA DO POVO, 711 DANFEDATA DE RECEBIMENTO IDENTIFICAÇÃO E ASSINATURA DO RECEBEDORNF-e
SÉRIE : 804000.793.272 Nº:
ELETRÔNICA
PROTOCOLO DE AUTORIZAÇÃO DE USO
141200000237323 - 05/03/2020 08:15:00
SEM VALOR FISCAL1 de 1804
FOLHA:
RECEBEMOS DE Teste NOME OS PRODUTOS/SERVIÇOS CONSTANTES NA NOTA FISCAL INDICADA AO LADO
jardim teste
MARINGA
PR
TEL/FAX: 4432222222
CEP: 79415000____/____/____
Consulta de autenticidade no portal nacional da NF-e 
www.nfe.fazenda.gov.br/portal 
ou no site da Sefaz AutorizadoraVALOR NOTA
R$ 0,01
DESTINATÁRIO
NF-E EMITIDA EM AMBIENTE DE HOMOLOGACAO - SEM VALOR FISCAL
NOME/RAZÃO SOCIAL CNPJ/CPF DATA DA EMISSÃO
CEP DATA DE SAÍDA/ENTR